In [43]:
import pandas as pd
import espn_api
import os
from dotenv import load_dotenv
from espn_api.football import League


load_dotenv()



True

In [44]:
print(os.getenv("LEAGUE_ID"))
print(2024)
print(os.getenv("ESPN_S2"))
print(os.getenv("SWID"))

582924
2024
AEA80I6JzZ%2F7RCxCSHl5WLd%2Bsvc7Xhs27An2hd4%2F1SSpMaMecJ6pq65wMTxW4AkykUz3MYAiBUBPM4IeLvlCfw4XBVJNSF96ilC4QnmIfLz9kJqsVNKNsqcOKR%2BdF0yjOdUWrmrT5EYJOxiho21V7NhB5qKxo5hyfagAQnEVP6GwZwbi9plSPWSuhxFEy%2FFm6eL1VgVnqTl1riAjrWyRo1zTBzGfrjgR1Oh%2B%2FEzHp%2FRAFHmy%2F0pAbsBzZtNU1DfWX%2FslmvtSRyg0yU8KLeOnuafM
{8771E5F0-2C1B-423A-AC82-A42344F84FDB}


In [3]:
league2024 = League(league_id=int(os.getenv("LEAGUE_ID")), year=2024, espn_s2=os.getenv("ESPN_S2"), swid=os.getenv("SWID"))
league2024.refresh()

In [4]:
teams2024 = league2024.teams
print(teams2024)
teams2024[0].stats

[Team(Hos'er Daddy), Team(Los Cocineros), Team(Purdy Commish), Team(xxx360NoScope420BlazeItxxx), Team(Sailor Jupiter Fan Club), Team(Fond O' Naps), Team(Biz-Caj Tanktop), Team(The Tortured Welfare Department), Team(San Francisco Cyber Plebz), Team(USA TCupz Dozzer), Team(LA Peezy), Team(SF Sheldors)]


{'defensiveLessThan100YardsAllowed': 0.0,
 'defensive100To199YardsAllowed': 2.0,
 'defensive200To299YardsAllowed': 6.0,
 'passingTouchdowns': 26.0,
 'defensive350To399YardsAllowed': 1.0,
 '5': 728.0,
 'defensive400To449YardsAllowed': 2.0,
 'defensive450To499YardsAllowed': 0.0,
 '198': 3.0,
 'defensive500To549YardsAllowed': 0.0,
 'lostFumbles': 5.0,
 'defensive550PlusYardsAllowed': 0.0,
 'madeFieldGoalsFrom60Plus': 1.0,
 'madeFieldGoalsFrom40To49': 6.0,
 'madeFieldGoalsFromUnder40': 20.0,
 'passing2PtConversions': 1.0,
 'passingInterceptions': 7.0,
 'missedFieldGoals': 5.0,
 'madeExtraPoints': 43.0,
 'rushingYards': 1936.0,
 'rushingTouchdowns': 29.0,
 'defensive0PointsAllowed': 0.0,
 'rushing2PtConversions': 0.0,
 'defensive1To6PointsAllowed': 1.0,
 'defensive7To13PointsAllowed': 3.0,
 'defensive14To17PointsAllowed': 4.0,
 'defensiveBlockedKickForTouchdowns': 0.0,
 'defensiveInterceptions': 13.0,
 'defensiveFumbles': 11.0,
 'defensiveBlockedKicks': 1.0,
 'defensiveSafeties': 0.0,
 'def

In [5]:
# print(teams2024[0])
# print(teams2024[0].scores)
# print(teams2024[0].owners)

In [6]:
teams = []
owner = []

for i in range(len(league2024.teams)):
  teams.append(teams2024[i].team_name)
  owner_info = teams2024[i].owners
  first_name = owner_info[0]['firstName']
  last_name = owner_info[0]['lastName']
  owner.append(first_name + " " + last_name)

print(teams)
print(owner)

["Hos'er Daddy", 'Los Cocineros', 'Purdy Commish', 'xxx360NoScope420BlazeItxxx', 'Sailor Jupiter Fan Club', "Fond O' Naps", 'Biz-Caj Tanktop', 'The Tortured Welfare Department', 'San Francisco Cyber Plebz', 'USA TCupz Dozzer', 'LA Peezy', 'SF Sheldors']
['Tim harris', 'Gabe Pannell', 'Brandon Lohmann', 'Eric Harvey', 'Edward Haven', 'Marco Aurelio Benites', 'Geoffrey Harris', 'Dennis Weber', 'Tom Weber', 'Raphael Dozzi', 'Andreas Panosyan', 'Alex E Sheldon']


In [7]:
#Generate all scores by team. The output list is that team's score per week
all_scores = []

for i in range(len(league2024.teams)):
  scores_by_team = []
  for j in range(13):
    scores_by_team.append(teams2024[i].scores[j])
  
  all_scores.append(scores_by_team)

print('scores - Tim', all_scores[0])

scores - Tim [92.5, 117.2, 114.7, 63.0, 112.5, 110.0, 110.8, 110.6, 131.8, 74.9, 127.6, 112.4, 110.5]


In [8]:
'''
Note: 
For each score:
1. Index each score into a tuple
2. Sort scores in ascending order by value
3. Cut off the top scores and insert the indices into a set
4. Take the original scores_by_team set and create the 1/-1 tuple
5. Provide that output
'''

num_weeks = len(all_scores[0])

# Accumulator for 
results = [[] for _ in range(len(league2024.teams))]


for week in range(num_weeks):

  #Transpose data to score by index of each time for a given week
  scores_with_index = [(i,all_scores[i][week]) for i in range(len(league2024.teams))]
  # print(scores_with_index)
  
  #Sort the data by score in ascending order; i.e. index 1
  scores_with_index.sort(key = lambda x: x[1], reverse=True)

  #Find the indices with the top scores
  cutoff = len(scores_with_index) // 2
  top_scores = scores_with_index[:cutoff]
  top_score_indices = {top_scores[i][0] for i in range(len(top_scores))}

  #Rebuild data with handicap
  for i in range(len(league2024.teams)):
    score = all_scores[i][week]
    result = +1 if i in top_score_indices else -1
    results[i].append((score,result))

In [9]:
# Original output by team/owner/weekly score
df = pd.DataFrame(all_scores,columns=[f"Week {i+1}" for i in range(13)])
df.insert(0,"Team",teams)
df.insert(0,"Owner",owner)
print(df)

                    Owner                             Team  Week 1  Week 2  \
0              Tim harris                     Hos'er Daddy    92.5   117.2   
1            Gabe Pannell                    Los Cocineros    95.9    76.0   
2         Brandon Lohmann                    Purdy Commish   132.6    75.4   
3             Eric Harvey       xxx360NoScope420BlazeItxxx    74.6   125.1   
4            Edward Haven          Sailor Jupiter Fan Club    83.0   129.3   
5   Marco Aurelio Benites                     Fond O' Naps   104.7    76.3   
6         Geoffrey Harris                  Biz-Caj Tanktop   121.1   102.3   
7            Dennis Weber  The Tortured Welfare Department    96.4   139.8   
8               Tom Weber        San Francisco Cyber Plebz   130.4   133.0   
9           Raphael Dozzi                 USA TCupz Dozzer    88.6   115.8   
10       Andreas Panosyan                         LA Peezy    85.1    89.0   
11         Alex E Sheldon                      SF Sheldors   115

In [10]:
# Results 
df = pd.DataFrame(results,columns=[f"Week {i+1}" for i in range(len(results[0]))])
df.insert(0,"Team",teams)
df.insert(0,"Owner",owner)

# df.to_csv('../data/processed/scores_with_handicap',index=False)
# df.to_excel('../data/processed/scores_with_handicap.xlsx')

In [11]:
print(df)

                    Owner                             Team      Week 1  \
0              Tim harris                     Hos'er Daddy  (92.5, -1)   
1            Gabe Pannell                    Los Cocineros  (95.9, -1)   
2         Brandon Lohmann                    Purdy Commish  (132.6, 1)   
3             Eric Harvey       xxx360NoScope420BlazeItxxx  (74.6, -1)   
4            Edward Haven          Sailor Jupiter Fan Club  (83.0, -1)   
5   Marco Aurelio Benites                     Fond O' Naps  (104.7, 1)   
6         Geoffrey Harris                  Biz-Caj Tanktop  (121.1, 1)   
7            Dennis Weber  The Tortured Welfare Department   (96.4, 1)   
8               Tom Weber        San Francisco Cyber Plebz  (130.4, 1)   
9           Raphael Dozzi                 USA TCupz Dozzer  (88.6, -1)   
10       Andreas Panosyan                         LA Peezy  (85.1, -1)   
11         Alex E Sheldon                      SF Sheldors  (115.3, 1)   

         Week 2       Week 3      Wee

In [12]:
wins_adj =[sum(1 for _,r in row if r == 1) for row in results]
losses_adj = [sum(1 for _,r in row if r == -1) for row in results]

print(wins_adj)
print(losses_adj)

[9, 3, 7, 7, 5, 6, 9, 9, 6, 3, 7, 7]
[4, 10, 6, 6, 8, 7, 4, 4, 7, 10, 6, 6]


In [13]:
df["+"]=wins_adj
df["-"]=losses_adj
print(df)

                    Owner                             Team      Week 1  \
0              Tim harris                     Hos'er Daddy  (92.5, -1)   
1            Gabe Pannell                    Los Cocineros  (95.9, -1)   
2         Brandon Lohmann                    Purdy Commish  (132.6, 1)   
3             Eric Harvey       xxx360NoScope420BlazeItxxx  (74.6, -1)   
4            Edward Haven          Sailor Jupiter Fan Club  (83.0, -1)   
5   Marco Aurelio Benites                     Fond O' Naps  (104.7, 1)   
6         Geoffrey Harris                  Biz-Caj Tanktop  (121.1, 1)   
7            Dennis Weber  The Tortured Welfare Department   (96.4, 1)   
8               Tom Weber        San Francisco Cyber Plebz  (130.4, 1)   
9           Raphael Dozzi                 USA TCupz Dozzer  (88.6, -1)   
10       Andreas Panosyan                         LA Peezy  (85.1, -1)   
11         Alex E Sheldon                      SF Sheldors  (115.3, 1)   

         Week 2       Week 3      Wee

In [14]:
wins = []
losses = []

for i in range(len(teams2024)):
  wins.append(teams2024[i].wins)
  losses.append(teams2024[i].losses)

print(wins)
print(losses)

[10, 5, 7, 7, 7, 9, 5, 9, 8, 3, 7, 7]
[4, 9, 7, 7, 7, 5, 9, 5, 6, 11, 7, 7]


In [15]:
df['wins']=wins
df['losses']=losses
print(df)

                    Owner                             Team      Week 1  \
0              Tim harris                     Hos'er Daddy  (92.5, -1)   
1            Gabe Pannell                    Los Cocineros  (95.9, -1)   
2         Brandon Lohmann                    Purdy Commish  (132.6, 1)   
3             Eric Harvey       xxx360NoScope420BlazeItxxx  (74.6, -1)   
4            Edward Haven          Sailor Jupiter Fan Club  (83.0, -1)   
5   Marco Aurelio Benites                     Fond O' Naps  (104.7, 1)   
6         Geoffrey Harris                  Biz-Caj Tanktop  (121.1, 1)   
7            Dennis Weber  The Tortured Welfare Department   (96.4, 1)   
8               Tom Weber        San Francisco Cyber Plebz  (130.4, 1)   
9           Raphael Dozzi                 USA TCupz Dozzer  (88.6, -1)   
10       Andreas Panosyan                         LA Peezy  (85.1, -1)   
11         Alex E Sheldon                      SF Sheldors  (115.3, 1)   

         Week 2       Week 3      Wee

In [16]:
df['adj_wins']=df['+'] + df['wins']
df['adj_losses']=df['-'] + df['losses']
print(df)
df.to_excel('../data/processed/scores_with_handicap.xlsx')

                    Owner                             Team      Week 1  \
0              Tim harris                     Hos'er Daddy  (92.5, -1)   
1            Gabe Pannell                    Los Cocineros  (95.9, -1)   
2         Brandon Lohmann                    Purdy Commish  (132.6, 1)   
3             Eric Harvey       xxx360NoScope420BlazeItxxx  (74.6, -1)   
4            Edward Haven          Sailor Jupiter Fan Club  (83.0, -1)   
5   Marco Aurelio Benites                     Fond O' Naps  (104.7, 1)   
6         Geoffrey Harris                  Biz-Caj Tanktop  (121.1, 1)   
7            Dennis Weber  The Tortured Welfare Department   (96.4, 1)   
8               Tom Weber        San Francisco Cyber Plebz  (130.4, 1)   
9           Raphael Dozzi                 USA TCupz Dozzer  (88.6, -1)   
10       Andreas Panosyan                         LA Peezy  (85.1, -1)   
11         Alex E Sheldon                      SF Sheldors  (115.3, 1)   

         Week 2       Week 3      Wee

In [17]:
# A manual validation using Ho'ser Daddy's information
print(df.loc[0])
print(all_scores[0])

Owner           Tim harris
Team          Hos'er Daddy
Week 1          (92.5, -1)
Week 2          (117.2, 1)
Week 3          (114.7, 1)
Week 4          (63.0, -1)
Week 5         (112.5, -1)
Week 6          (110.0, 1)
Week 7          (110.8, 1)
Week 8          (110.6, 1)
Week 9          (131.8, 1)
Week 10         (74.9, -1)
Week 11         (127.6, 1)
Week 12         (112.4, 1)
Week 13         (110.5, 1)
+                        9
-                        4
wins                    10
losses                   4
adj_wins                19
adj_losses               8
Name: 0, dtype: object
[92.5, 117.2, 114.7, 63.0, 112.5, 110.0, 110.8, 110.6, 131.8, 74.9, 127.6, 112.4, 110.5]


In [18]:
print(teams2024[0].wins)
print(teams2024[0].losses)

10
4


In [19]:
week1 = df['Week 1']
week1_list = df['Week 1'].tolist()
indexed_list = []

for i in range(len(week1_list)):
  indexed_list.append((i,week1_list[i]))

indexed_list.sort(key=lambda x: x[1],reverse=True)
print(indexed_list)


[(2, (132.6, 1)), (8, (130.4, 1)), (6, (121.1, 1)), (11, (115.3, 1)), (5, (104.7, 1)), (7, (96.4, 1)), (1, (95.9, -1)), (0, (92.5, -1)), (9, (88.6, -1)), (10, (85.1, -1)), (4, (83.0, -1)), (3, (74.6, -1))]


In [20]:
cutoff = len(week1_list) // 2
print('indexed_list',indexed_list)

#Create a set and then check whether value for the week is in the set
top_indices = {idx for idx, _ in indexed_list[:cutoff]}
print('top_indices',top_indices)


indexed_list [(2, (132.6, 1)), (8, (130.4, 1)), (6, (121.1, 1)), (11, (115.3, 1)), (5, (104.7, 1)), (7, (96.4, 1)), (1, (95.9, -1)), (0, (92.5, -1)), (9, (88.6, -1)), (10, (85.1, -1)), (4, (83.0, -1)), (3, (74.6, -1))]
top_indices {2, 5, 6, 7, 8, 11}


In [21]:
list_with_handicap = []
for i in range(len(week1_list)):
  if(i in top_indices):
    list_with_handicap.append((week1_list[i],1))
  else:
    list_with_handicap.append((week1_list[i],-1))

print(list_with_handicap)

[((92.5, -1), -1), ((95.9, -1), -1), ((132.6, 1), 1), ((74.6, -1), -1), ((83.0, -1), -1), ((104.7, 1), 1), ((121.1, 1), 1), ((96.4, 1), 1), ((130.4, 1), 1), ((88.6, -1), -1), ((85.1, -1), -1), ((115.3, 1), 1)]


In [22]:
teams2024 = league2024.teams
# print(teams2024)
# teams2024[0].stats

data = {}
data['team'] = []
data['stat'] = []
data['result'] = []

for i in range(len(teams2024)):
  current_team = teams2024[i]

  for key in current_team.stats:
    data['team'].append(current_team.team_name)
    if(key == '5'):
      data['stat'].append('passingYards')
      data['result'].append(current_team.stats[key])
    elif(key == '198'):
      data['stat'].append('defensiveFumblesRecovered')
      data['result'].append(current_team.stats[key])
    else:
      data['stat'].append(key)
      data['result'].append(current_team.stats[key])

df_stats = pd.DataFrame(data)
print(df_stats)

df_stats.to_excel('../data/processed/stats_2024.xlsx')

             team                              stat  result
0    Hos'er Daddy  defensiveLessThan100YardsAllowed     0.0
1    Hos'er Daddy     defensive100To199YardsAllowed     2.0
2    Hos'er Daddy     defensive200To299YardsAllowed     6.0
3    Hos'er Daddy                 passingTouchdowns    26.0
4    Hos'er Daddy     defensive350To399YardsAllowed     1.0
..            ...                               ...     ...
523   SF Sheldors               receivingReceptions   301.0
524   SF Sheldors      defensive28To34PointsAllowed     2.0
525   SF Sheldors      defensive35To45PointsAllowed     2.0
526   SF Sheldors      defensive45PlusPointsAllowed     0.0
527   SF Sheldors              fumbleRecoveredForTD     0.0

[528 rows x 3 columns]


In [23]:
teams2024 = league2024.teams
# print(teams2024)
# teams2024[0].stats

data = {}
data['stat'] = []
data['result'] = []

writer = pd.ExcelWriter("../data/processed/stats_by_team_2024.xlsx", engine="openpyxl")

for i in range(len(teams2024)):
  data = {}
  data['stat'] = []
  data['result'] = []
  current_team = teams2024[i]

  for key in current_team.stats:
    if(key == '5'):
      data['stat'].append('passingYards')
      data['result'].append(current_team.stats[key])
    elif(key == '198'):
      data['stat'].append('defensiveFumblesRecovered')
      data['result'].append(current_team.stats[key])
    else:
      data['stat'].append(key)
      data['result'].append(current_team.stats[key])
  
  df_stats = pd.DataFrame(data)
  df_stats.to_excel(writer,sheet_name=current_team.team_name,index=False)

writer.close()


In [24]:
league2025 = League(league_id=int(os.getenv("LEAGUE_ID")), year=2025, espn_s2=os.getenv("ESPN_S2"), swid=os.getenv("SWID"))

In [46]:
print(league2024.power_rankings(1))
print(league2024.teams[0].owners[0]['firstName'] + ' ' + league2024.teams[0].owners[0]['lastName'])
print(teams)

[('22.95', Team(Purdy Commish)), ('22.35', Team(San Francisco Cyber Plebz)), ('20.15', Team(Biz-Caj Tanktop)), ('19.00', Team(SF Sheldors)), ('17.00', Team(Fond O' Naps)), ('13.65', Team(Sailor Jupiter Fan Club)), ('13.30', Team(Los Cocineros)), ('13.20', Team(Hos'er Daddy)), ('13.20', Team(The Tortured Welfare Department)), ('11.15', Team(USA TCupz Dozzer)), ('10.70', Team(xxx360NoScope420BlazeItxxx)), ('10.40', Team(LA Peezy))]
Tim harris
["Hos'er Daddy", 'Los Cocineros', 'Purdy Commish', 'xxx360NoScope420BlazeItxxx', 'Sailor Jupiter Fan Club', "Fond O' Naps", 'Biz-Caj Tanktop', 'The Tortured Welfare Department', 'San Francisco Cyber Plebz', 'USA TCupz Dozzer', 'LA Peezy', 'SF Sheldors']


In [ ]:
week = 1
weekly_scoreboard = dict()
weekly_scoreboard['week'] = week
weekly_scoreboard['year'] = league.year
print(weekly_scoreboard)

scoreboard_by_week = league.scoreboard(week)
print(scoreboard_by_week[0].home_team.team_name)

#home_team
#home_score
#away_team
#away_score
#opposing_team
#is_home_team (bool)

# for scoreboard in scoreboard_by_week:


{'week': 1, 'year': 2024}
Hos'er Daddy


In [56]:
scoreboard_2024=league2024.scoreboard()

# for team in teams2024:
#   print(team.streak_type)
#   print(team.streak_length)

league2025.power_rankings(2)

[('0.00', Team(Hos'er Daddy)),
 ('0.00', Team(Los Cocineros)),
 ('0.00', Team(Purdy Commish)),
 ('0.00', Team(xxx360NoScope420BlazeItxxx)),
 ('0.00', Team(Cloe From 4th and Long)),
 ('0.00', Team(Fond O' Naps)),
 ('0.00', Team(Biz-Caj Tanktop)),
 ('0.00', Team(Blitz Me Like You Missed Me)),
 ('0.00', Team(San Francisco Cyber Plebz)),
 ('0.00', Team(USA TCupz Dozzer)),
 ('0.00', Team(LA Peezy)),
 ('0.00', Team(SF Sheldors))]